In [1]:
import time

start_time = time.time()

**Εγκατάσταση Java**

In [2]:
#Εγκατάσταση Java για να δουλέψει το Spark
!apt-get update
!apt-get install openjdk-17-jdk -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,830 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,290 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/ma

**Εκκίνηση Spark Session**

In [3]:
!pip -q install pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "4g").getOrCreate()
sc = spark.sparkContext
print("Spark Session Started")

Spark Session Started


**RDD File, Split/Parse**

In [ ]:
rdd = sc.textFile('flights_2000.csv')

header = rdd.first() #Για να φιλτράρουμε τα headers απο το υπόλοιπο αρχείο
rdd = rdd.filter(lambda line: line != header)

rdd_split = rdd.map(lambda line: line.split(','))

# Συνάρτηση ώστε να κάνουμε parse μόνο τα απαραίτητα fields
def parse_origin_delay(line_list):
    origin_airport = line_list[3] # Το κρατάμε ως string
    # Μετατρέπουμε το dep_delay σε int εκτός αν είναι κενό
    dep_delay_str = line_list[6]
    dep_delay = int(dep_delay_str) if dep_delay_str else None 

    return (origin_airport, dep_delay)

rdd_origin_delay = rdd_split.map(parse_origin_delay)

**Υπολογισμός μέσης καθυστέρησης και εκτύπωση**

In [5]:
# Κάνουμε filter τα None values επειδή δεν μπορούν να υπολογιστούν στον μέσο όρο
rdd_filtered_delay = rdd_origin_delay.filter(lambda x: x[1] is not None)

# Δημιουργούμε τα functions για το combineByKey
createCombiner = lambda dep_delay: (dep_delay, 1)

mergeValue = lambda combiner, dep_delay: (combiner[0] + dep_delay, combiner[1] + 1)

mergeCombiners = lambda comb1, comb2: (comb1[0] + comb2[0], comb1[1] + comb2[1])

# Εφαρμογή του combineByKey
rdd_sum_count = rdd_filtered_delay.combineByKey(
    createCombiner,
    mergeValue,
    mergeCombiners
)

rdd_avg_delay = rdd_sum_count.mapValues(lambda sum_count: sum_count[0] / sum_count[1])

#Ταξινόμηση
action_start_time = time.time() # Έναρξη χρονομέτρησης για το action
top_10_airports = rdd_avg_delay.sortBy(lambda x: x[1], ascending=False).take(10)
action_end_time = time.time() # Τέλος χρονομέτρησης για το action

print("Top 10 Origin Airports with Highest Average Departure Delay:")
for airport, avg_delay in top_10_airports:
    print(f"{airport}: {avg_delay:.2f}")

print(f"\nTime taken for the first Spark action (take(10)): {action_end_time - action_start_time:.2f} seconds")

end_time = time.time()
print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Top 10 Origin Airports with Highest Average Departure Delay:
DFW: 11.98
JFK: 11.20
SEA: 9.56
ORD: 9.35
LAS: 9.11
CLT: 8.82
MCO: 8.69
BOS: 8.63
DEN: 8.44
MIA: 8.26

Time taken for the first Spark action (take(10)): 7.97 seconds

Total execution time: 54.99 seconds
